In [2]:
## I am going to use the swagger API client for cbioportal to generate some basic folders structure
## for the data set
import girder_client
import tqdm,re,sys
import girder_utils as gu

dccGC = girder_client.GirderClient(apiUrl="https://imaging.htan.dev/girder/api/v1")
from bravado.client import SwaggerClient
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/api-docs',
                                    config={"validate_requests":False,"validate_responses":False})
#https://docs.cbioportal.org/6.-web-api-and-clients/api-and-api-clients
dccGC.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '5ebee66965f992910fa19570',
 '_modelType': 'user',
 'admin': True,
 'created': '2020-05-15T18:58:49.562000+00:00',
 'email': 'girder.admin@kitware.com',
 'emailVerified': True,
 'firstName': 'admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 167707610500,
 'status': 'enabled'}

In [3]:
## These are the main tcga groups not subdivided based on publications and such
tcgaRootCancerTypes = ['acc_tcga', 'cesc_tcga', 'chol_tcga','esca_tcga', 'gbm_tcga','hnsc_tcga',
 'kirp_tcga', 'lusc_tcga', 'meso_tcga','thym_tcga', 'thca_tcga', 'ucs_tcga', 'uvm_tcga',
 'skcm_tcga', 'paad_tcga', 'pcpg_tcga', 'prad_tcga', 'stad_tcga', 'tgct_tcga', 'kirc_tcga', 'dlbc_tcga', 
 'lihc_tcga', 'luad_tcga', 'ov_tcga', 'ucec_tcga', 'brca_tcga', 'kich_tcga']
# for a in dir(cbioportal):
#     cbioportal.__setattr__(a.replace(' ', '_').lower(), cbioportal.__getattr__(a))

In [ ]:
# for ct in cbioportal.Cancer_Types.getAllCancerTypesUsingGET().result():
#     print(ct)
cbioportal.Clinical_Data.getAllClinicalDataOfPatientInStudyUsingGET(patientId='TCGA-02-0006',studyId='gbm_tcga').result()

In [7]:
ptMetaData = {}

for studyId in tcgaRootCancerTypes:
    print("Processing %s" % studyId)
    ptMetaData[studyId] = {}
    studyClinData = cbioportal.Clinical_Data.getAllClinicalDataInStudyUsingGET(studyId=studyId).result()
    
    for r in studyClinData:
        patientId = r['patientId']
        if patientId not in ptMetaData[studyId]:
            ptMetaData[studyId][patientId] = {}
        ptMetaData[studyId][patientId][r['clinicalAttributeId']] = r['value']

Processing acc_tcga
Processing cesc_tcga
Processing chol_tcga
Processing esca_tcga
Processing gbm_tcga
Processing hnsc_tcga
Processing kirp_tcga
Processing lusc_tcga
Processing meso_tcga
Processing thym_tcga
Processing thca_tcga
Processing ucs_tcga
Processing uvm_tcga
Processing skcm_tcga
Processing paad_tcga
Processing pcpg_tcga
Processing prad_tcga
Processing stad_tcga
Processing tgct_tcga
Processing kirc_tcga
Processing dlbc_tcga
Processing lihc_tcga
Processing luad_tcga
Processing ov_tcga
Processing ucec_tcga
Processing brca_tcga
Processing kich_tcga


In [ ]:
tcgaStudyIdList = []

for sl in cbioportal.Sample_Lists.getAllSampleListsUsingGET().result():
    if sl.category == 'all_cases_in_study' and 'tcga' in sl.studyId:
        ### Going to do a subfilter as well... I want all the samples not just the ones used in the pub
        if 'pub' not in sl.studyId:  ## Other logic is to use ends_with "all"
            tcgaStudyIdList.append(sl.studyId)       
## Want category=='all_cases_in_study'

In [ ]:
## I can add an offest if I want to this call
class LinePrinter():
    """
    Print things to stdout on one line dynamically
    """
    def __init__(self,data):
        sys.stdout.write("\r\x1b[K"+data.__str__())
        sys.stdout.flush()
        
def getAllItems( client, folderOrCollectionId, parentType='folder', batchSize = 1000, offset=0, debug=False ):
    # client is the girder client Object
    # I am making this more generic, so I can pass a folder or collection Id
    # This now returns a generator so should be more efficient
#     itemSet = []
    limit = batchSize
    batchesProcessed = 0
    colId = folderOrCollectionId

    getStr = "resource/%s/items?limit=%d&offset=%d&type=%s" % (colId,limit,offset,parentType) 
    #if debug: print(getStr)
    gcImageList = client.get( getStr)

    while gcImageList:
        for i in gcImageList:
            yield(i)
    batchesProcessed +=1
#         itemSet.extend(gcImageList)

        getStr = "resource/%s/items?limit=%d&offset=%d&type=%s" % ( colId,batchSize,(int(batchSize)*int(batchesProcessed)),parentType) 
        if debug:
            output = "Processed %d items" % (int(batchSize)*int(batchesProcessed))
            LinePrinter(output)
        gcImageList = client.get( getStr)  

In [11]:
tcgaCollectionId = '5ebee678aca0c580df115376'
import tqdm
for studyId in tcgaRootCancerTypes:
    print("Processing data for %s" % studyId)
    studyIdFolder = dccGC.createFolder(tcgaCollectionId,studyId,parentType='collection',reuseExisting=True)
    
    for subj in tqdm.tqdm(ptMetaData[studyId]):
        subjFolder = dccGC.createFolder(studyIdFolder['_id'],subj,reuseExisting=True)
        dccGC.addMetadataToFolder(subjFolder['_id'],ptMetaData[studyId][subj])
        

Processing data for acc_tcga


  0%|          | 0/308 [00:00<?, ?it/s]

Processing data for cesc_tcga


  0%|          | 0/51 [00:00<?, ?it/s]

Processing data for chol_tcga


...

Processing data for gbm_tcga


  0%|          | 0/528 [00:00<?, ?it/s]

Processing data for hnsc_tcga


 57%|█████▋    | 299/528 [01:32<01:13,  3.10it/s]

ConnectionError: HTTPSConnectionPool(host='imaging.htan.dev', port=443): Max retries exceeded with url: /girder/api/v1/folder/5fc5b5faa852689f133b90fb/metadata (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4e2591e908>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
https://www.cbioportal.org/api/studies/gbm_tcga/patients/TCGA-02-0006/clinical-data?direction=ASC&pageNumber=0&pageSize=10000000&projection=SUMMARY

### Need to add this 
    
# tcga2OpenFolderId = '5fbc102f79c054aa9327869e'
# matchTCGAsvs = re.compile(r'(?P<tcgaID>TCGA-..-....).*(?P<ext>svs)')


# tcgaSubjFolderCache = {}
# dccGC.authenticate(interactive=True)

In [ ]:
tgtTCGARootFolder = '5fc2f483a852689f133b147d'

#for i in getAltqdm.tqdm(dccGC.listResource('resource/%s/items?type=folder' % tcga2OpenFolderId)):
for i in getAllItems(dccGC,tcga2OpenFolderId,offset=81000,debug=True):
    m = matchTCGAsvs.search(i['name'])
    if m:
        tcgaID = m.groupdict()['tcgaID']
        if tcgaID not in tcgaSubjFolderCache:
            tcgaSubjFolder = dccGC.createFolder(tgtTCGARootFolder,tcgaID,reuseExisting=True)
            tcgaSubjFolderCache[tcgaID] = tcgaSubjFolder
    
        tcgaSubjFolder = tcgaSubjFolderCache[tcgaID]    
        ### ADD A CHECK TO AVOID COPYING FILES THAT ARE ALREADY IN THERE

        itemsInFolder = list( dccGC.listItem(tcgaSubjFolder['_id']))
        
        if i['name'] not in [x['name'] for x in itemsInFolder]:
            dccGC.post(f"/item/{i['_id']}/copy?folderId={tcgaSubjFolder['_id']}")